In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [2]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"


In [3]:
response = requests.get(url)
soup = bs(response.text, "html.parser")
soup


<!DOCTYPE html>

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- Always force latest IE rendering engine or request Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5e33925808","applicationID":"59562082","transactionName":"JVcPR0MLWApSRU1eAQVVEhxSC1oSUlkWbBMHXwRAHhdcCUA=","queueTime":0,"applicationTime":220,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQcPUlZTDxAFXVRUBQEPVA=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o

## NASA Mars News

In [4]:
title_result = soup.find("div", class_="content_title")
news_title = title_result.a.text.strip()

paragraph_result = soup.find("div", class_="rollover_description_inner")
news_p = paragraph_result.text.strip()

In [5]:
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"


In [6]:
!which chromedriver


/usr/local/bin/chromedriver


In [7]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(image_url)

In [8]:
html = browser.html
soup = bs(html, 'html.parser')
imgBase_url = 'https://www.jpl.nasa.gov'

img_partial_url = soup.find("a", class_="fancybox")["data-fancybox-href"]

## Mars Space Images

In [9]:
img_url = f'{imgBase_url}{img_partial_url}'
img_url


'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16028_ip.jpg'

In [10]:
mars_weather_url = "https://twitter.com/marswxreport?lang=en"
response = requests.get(mars_weather_url)
soup = bs(response.text, "html.parser")
soup

<!DOCTYPE html>

<html data-scribe-reduced-action-queue="true" lang="en">
<head>
<meta charset="utf-8"/>
<script nonce="AfYsVhHAXerGpcUJs1actw==">
        !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")>-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();
      </script>
<script id="bouncer_terminate_iframe" nonce="AfYsVhHAXerGpcUJs1actw==">
    if (window.top != window) {
  window.top.postMessage({'bouncer': true, 'event': 'complete'}, '*');
}
  </script>
<script id="swift_action_queue" nonce="AfYsVhHAXerGpcUJs1actw==">
    !function(){function e(e){if(e||(e=window.event),!e)return!1;if(e.timestamp=(new Date).getTime(),!e.target&&e.srcElement&&(e.target=e.srcElement),document.documentElement.getAttribute("data-scribe-reduced-action-queue"))for(var t=e.target;t&&t!=document.body;){if("A"==t.tagName)return;t=t.parentNode}return i("all",o(e)),a(e)?(document.addEventListener||(e

## Mars Weather

In [11]:
tweets = soup.find_all("div",class_="content")

for tweet in tweets:
    if tweet.b.text == "MarsWxReport":
        curr_weather = soup.find("p", class_="TweetTextSize").text.replace('\n',' ')
        break
        
curr_weather


'InSight sol 182 (2019-06-01) low -99.7ºC (-147.5ºF) high -22.6ºC (-8.7ºF) winds from the SSE at 5.7 m/s (12.7 mph) gusting to 15.9 m/s (35.5 mph) pressure at 7.50 hPapic.twitter.com/NmzIAqOiDG'

In [12]:
table_url = "https://space-facts.com/mars/"
table_list = pd.read_html(table_url)
mars_fact_table = table_list[0]
mars_fact_table.columns = ["Description","Value"]
mars_fact_table


,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


## Mars Facts

In [13]:
mars_fact_html_table = mars_fact_table.to_html()
mars_fact_html_table

mars_table_dict = {}

mars_table_dict["table_code"]=mars_fact_html_table

## Hemisphere Images

In [14]:
hemisphere_image_urls = []
hemisphere_image_links = ("https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced",
"https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced",
"https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced",
"https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced")

for url in hemisphere_image_links:
    response = requests.get(url)
    soup = bs(response.text, "html.parser")
    title = soup.find("h2", class_="title").text
    pic_url = soup.find("a",target="_blank")["href"]
    
    temp_dict = {}
    temp_dict["Title"]=title
    temp_dict["img_url"]=pic_url
    hemisphere_image_urls.append(temp_dict)
    temp_dict={}
    
hemisphere_image_urls


[{'Title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]